In [1]:
import urllib.request
from bs4 import BeautifulSoup as bs
import re
import pandas as pd 

In [2]:
def data_rbref_df(a,year,b,category,c,number_of_headers):
    url=(a+year+b+category+c)
    page = urllib.request.urlopen(url)
    soup = bs(page)
    td_s = soup.body.findAll('td') # targets the data table at the url page 
    data_cells = (str(td_s)).split(',') # gives me each data cell as an index in a list 
    
    table_header_temp=[]
    table_data_temp=[]
    table_header=[]
    table_data=[]

    #find in the html text data headers and data itself 
    for cells in data_cells:
        table_header_temp.append(re.findall('data-stat="\w+"', cells))
        table_data_temp.append(re.findall('>(.*?)<', cells))

    table_header = [(str(item).split('=')[1][1:-3]) for item in table_header_temp]

    for i in table_data_temp:
        table_data.append(''.join(s for s in i if s)) # omit empty items in list


    keys_header = table_header[0:number_of_headers]
    values_data = [[] for i in range(len(keys_header))]
    number_of_games=38
   

    for i in range(len(keys_header)):
        for j in range(number_of_games):
            values_data[i].append(table_data[i+j*len(keys_header)])

    my_dict = dict(zip(keys_header, values_data))
    df = pd.DataFrame.from_dict(my_dict)
    df = df.drop(columns=['match_report'])
    return df

def df_5_seasons_perteam(team, db_id):
    
    cat=['keeper', 'defense', 'possession', 'passing', 'passing_types', 'gca', 'shooting', 'misc' ]
    num_headers_percat=[35,32,34,30,34,23,24,25] #17 for schedule
    years=['2017-2018', '2018-2019', '2019-2020', '2020-2021', '2021-2022']
    
    a=('https://fbref.com/en/squads'+ db_id)
    b='/matchlogs/c12/'
    c=('//'+team+'-Match-Logs-La-Liga')

    # empty lists for temp calcs
    list_dfs = [[] for l in range(len(years))]
    temp_df = [[] for l in range(len(years))]
    df_temp = [[] for l in range(len(years))]

    # manipulate df of each year to create df for 2017-2022  
    for y in range(len(years)):
        for ca in range(len(cat)):
            list_dfs[y].append(data_rbref_df(a,years[y],b,cat[ca],c,num_headers_percat[ca]))
        temp_df[y] = pd.concat(list_dfs[y][1:], axis=1)
        temp_df[y] = temp_df[y].drop(columns=['start_time', 'round', 'dayofweek', 'venue', 'result', 'goals_for','goals_against', 'opponent'])
        df_temp[y] = pd.concat([list_dfs[y][0],temp_df[y]],axis=1)

    df = pd.concat(df_temp , axis=0)
    return df 

In [3]:
#I divided all the teams into three groups so that if a link is broken not all is gone 

teams = ['Sevilla', 'Athletic-Club', 'Barcelona', 'Celta-Vigo', 'Real-Madrid', 'Real-Sociedad'] 
cor_db_id = ['/ad2be733/','/2b390eca/', '/206d90db/', '/f25da7fb/', '/53a2f082/', '/e31d1cd9/']  #team's corresponding data_base id 
la_liga_df_temp1 = []

for t in range(len(teams)):
    la_liga_df_temp1.append(df_5_seasons_perteam(teams[t], cor_db_id[t]))

df_1 = pd.concat(la_liga_df_temp1, axis=0)
df_1.info()

                           


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1140 entries, 0 to 37
Columns: 173 entries, start_time to aerials_won_pct
dtypes: object(173)
memory usage: 1.5+ MB


In [4]:
teams = ['Valencia', 'Villarreal', 'Atletico-Madrid', 'Betis']
cor_db_id = ['/dcc91a7b/', '/2a8183b3/', '/db3b9613/', '/fc536746/']

la_liga_df_temp2 = []

for t in range(len(teams)):
    la_liga_df_temp2.append(df_5_seasons_perteam(teams[t], cor_db_id[t]))

df_2 = pd.concat(la_liga_df_temp2, axis=0)
df_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 760 entries, 0 to 37
Columns: 173 entries, start_time to aerials_won_pct
dtypes: object(173)
memory usage: 1.0+ MB


In [5]:
teams = ['Levante', 'Getafe', 'Alavs']
cor_db_id = ['/9800b6a1/', '/7848bd64/', '/8d6fd021/']

la_liga_df_temp3 = []

for t in range(len(teams)):
    la_liga_df_temp3.append(df_5_seasons_perteam(teams[t], cor_db_id[t]))

df_3 = pd.concat(la_liga_df_temp3, axis=0)
df_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 570 entries, 0 to 37
Columns: 173 entries, start_time to aerials_won_pct
dtypes: object(173)
memory usage: 774.8+ KB


In [6]:
df = pd.concat([df_1, df_2, df_3], axis=0)
df.info()
df.to_csv('la_liga_data.csv')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2470 entries, 0 to 37
Columns: 173 entries, start_time to aerials_won_pct
dtypes: object(173)
memory usage: 3.3+ MB


In [8]:
teams =['Sevilla', 'Athletic-Club', 'Barcelona', 'Celta-Vigo', 'Real-Madrid', 'Real-Sociedad', 'Valencia', 'Villarreal', 'Atletico-Madrid', 'Betis', 'Levante', 'Getafe', 'Alavs']
cor_db_id=['ad2be733','2b390eca', '206d90db', 'f25da7fb', '53a2f082', 'e31d1cd9', 'dcc91a7b', '2a8183b3', 'db3b9613', 'fc536746', '9800b6a1', '7848bd64', '8d6fd021']
